In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Ensure we are in the main project directory
import os

if not os.getcwd().endswith("chore-optimizer"):
    os.chdir("..")

print(f"Current working directory: {os.getcwd()}")

Current working directory: /Users/riegejon/repos/chore-optimizer


In [3]:
from ortools.sat.python import cp_model
import numpy as np
import pandas as pd

In [36]:
people = ["W", "R"]
num_people = len(people)
num_periods = 52

# Define chores (name, interval, workload)
chores = [
    ("Check fire extinguisher", 52, 1),
    ("Lubricate door hinges", 26, 2),
    ("Clean faucet aerators", 26, 2),
    ("Window maintenance", 26, 3),
    ("Clean lower cupboards", 26, 5),
    ("Clean upper cupboards", 26, 5),
    ("Clean under the sink", 13, 4),
    ("Clean bedroom windows", 6, 4),
    ("Clean living room windows", 6, 4),
    ("Clean oven", 6, 4),
    ("Clean fridge", 6, 4),
    ("Scrub shower", 6, 4),
    ("Wash hood filters", 3, 1),
    ("Clean microwave", 6, 2),
]
num_chores = len(chores)
chore_intervals = [c[1] for c in chores]
chore_workloads = [c[2] for c in chores]

# Weight of the objectives
max_workload_weight = 1.0
diff_workload_weight = 1.0

model = cp_model.CpModel()

# Create the boolean tensor of who is doing what chore in what period
# (person, chore, period)
assignments = {}
for p in range(num_people):
    for c in range(num_chores):
        for t in range(num_periods):
            assignments[(p, c, t)] = model.NewBoolVar(
                f"person_{p}_chore_{c}_period_{t}"
            )

# Criterion 1: Each chore must be performed at regular intervals
for c in range(num_chores):
    for t in range(num_periods - chore_intervals[c]):
        chore_performed_at_t = sum(
            assignments[(p, c, t)] for p in range(num_people)
        )
        chore_performed_at_t_plus_interval = sum(
            assignments[(p, c, t + chore_intervals[c])]
            for p in range(num_people)
        )
        model.Add(chore_performed_at_t == chore_performed_at_t_plus_interval)

# Criterion 2: Each chore must be performed at least N times and at most N + 1
# times, where N is the number of periods divided by the interval of the chore
for c in range(num_chores):
    n_times_chore_performed = sum(
        assignments[(p, c, t)]
        for p in range(num_people)
        for t in range(num_periods)
    )
    chore_times_to_perform_lower_bound = num_periods // chore_intervals[c]
    model.Add(n_times_chore_performed >= chore_times_to_perform_lower_bound)
    model.Add(n_times_chore_performed <= chore_times_to_perform_lower_bound + 1)

    # Special case for chores that can be performed at most once
    if chore_intervals[c] >= num_periods:
        model.Add(n_times_chore_performed == 1)

# Criterion 3: In a given period, the scheduled chores must each be performed by
# exactly one person
for c in range(num_chores):
    for t in range(num_periods):
        model.add_at_most_one(assignments[(p, c, t)] for p in range(num_people))

# Criterion 4: Each chore must be assigned approximately the same number of times
# to each person
for c in range(num_chores):
    for p in range(num_people):
        n_times_chore_assigned_to_person = sum(
            assignments[(p, c, t)] for t in range(num_periods)
        )
        n_times_minimum_per_person = num_periods // (
            num_people * chore_intervals[c]
        )
        model.Add(
            n_times_chore_assigned_to_person >= n_times_minimum_per_person
        )

# Criterion 5: Never assign the same to chore to the same person in consecutive
# intervals
for c in range(num_chores):
    for p in range(num_people):
        for t in range(num_periods - chore_intervals[c]):
            model.add_at_most_one(
                assignments[(p, c, t)],
                assignments[(p, c, t + chore_intervals[c])],
            )

# Symmetry breaking: assign the first chore to the first person in the first period
model.Add(assignments[(0, 0, 0)] == 1)

# Objective 1: Spread the workload as evenly as possible across periods by
# minimizing the difference between the workloads of each period and the
# average workload
total_workload = sum(
    (num_periods / chore_intervals[c]) * chore_workloads[c]
    for c in range(num_chores)
)
avg_workload_periods = round(total_workload / (num_people * num_periods))

workload_diff_across_periods = []
for p in range(num_people):
    for t in range(num_periods):
        workload = sum(
            (assignments[(p, c, t)] * chore_workloads[c])
            for c in range(num_chores)
        )
        workload_diff = model.NewIntVar(
            0, 999, f"workload_diff_person_{p}_period_{t}"
        )
        model.add_abs_equality(workload_diff, workload - avg_workload_periods)
        workload_diff_across_periods.append(workload_diff)

# Objective 2: Spread the workload as evenly as possible across people by
# minimizing the difference between the total workloads of each person and the
# average workload of each person
avg_workload_person = (
    sum(
        chore_times_to_perform[c] * chore_workloads[c]
        for c in range(num_chores)
    )
    // num_people
)
workload_diff_persons = []
for p in range(num_people):
    workload = sum(
        (assignments[(p, c, t)] * chore_workloads[c])
        for c in range(num_chores)
        for t in range(num_periods)
    )
    workload_diff = model.NewIntVar(0, 999, f"workload_diff_person_{p}")
    model.add_abs_equality(workload_diff, workload - avg_workload_person)
    workload_diff_persons.append(workload_diff)

model.minimize(sum(workload_diff_across_periods) + sum(workload_diff_persons))

In [40]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 200.0

In [45]:
# Register a callback to print the solution
class ChoresPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, chores, people, num_periods):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._chores = chores
        self._people = people
        self._num_chores = len(chores)
        self._num_people = len(people)
        self._num_periods = num_periods
        self._solution_count = 0

    def on_solution_callback(self):
        self._solution_count += 1
        print(
            f"Solution {self._solution_count} (obj. value {self.ObjectiveValue()}):"
        )

        periods = [str(t + 1) for t in range(self._num_periods)]
        chores = [c[0] for c in self._chores]
        df = pd.DataFrame("", index=chores, columns=periods)
        for c in range(self._num_chores):
            for t in range(self._num_periods):
                for p in range(self._num_people):
                    if self.Value(assignments[(p, c, t)]):
                        df.loc[self._chores[c][0], str(t + 1)] = self._people[p]

        print(df.to_string(col_space=2, justify="right") + "\n")

    def solutionCount(self):
        return self._solution_count


# Display the first five solutions.
solution_printer = ChoresPartialSolutionPrinter(chores, people, num_periods)

In [44]:
status = solver.solve(model, solution_printer)

print(f"Status: {solver.StatusName(status)}")

Solution 1 (obj. value 262.0):
                           1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
Check fire extinguisher    W                                                                                                                                                         
Lubricate door hinges                                                                  R                                                                             W               
Clean faucet aerators                                                      W                                                                             R                           
Window maintenance                                                            R                                                                             W                        
Clean lower cupboards                                      